In [ ]:
##pip install jupyter_contrib_nbextensions

#### Load data

In [41]:
#load libraries
import pandas as pd 
import numpy as np 
from datetime import datetime
import sys
import ast
import csv

#import nltk
#from nltk.corpus import stopwords
#import spacy

import matplotlib.pyplot as plt
import seaborn as sns



import networkx as nx
from networkx.algorithms.components.connected import connected_components

import json
import dask.bag as db

import sys
import os

sys.path.append("..")

from pathlib import Path

import json


from itertools import combinations
from collections import Counter
from itertools import chain
import random

from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations

import multiprocessing

from IPython.core.display import display, HTML

from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

from itertools import permutations
from pyspark import SparkConf, SparkContext, SQLContext
import pyspark
import sys
from collections import defaultdict
from graphframes import *

In [ ]:
## List of categories  http://arxitics.com/help/categories?group=cs

In [ ]:
"""'cs.AI','cs.CL','cs.CC','cs.CE','cs.CG','cs.GT','cs.CV','cs.CY','cs.CR','cs.DS'
           ,'cs.DB','cs.DL','cs.DM','cs.DC','cs.DT','cs.FL','cs.GL','cs.GR','cs.AR','cs.HC'
           ,'cs.IR','cs.IT','cs.LG','cs.LO','cs.MS','cs,MA','cs.MM','cs.NI','cs.NE','cs.NA'
           ,'cs.OS','cs.OH','cs.PF','cs.PL','cs.RO','cs.SI','cs.SE','cs.SD','cs.SC','cs.SY'"""

In [2]:
## set checkpoint
sc.stop()
if not os.path.isdir("checkpoints"):
    os.mkdir("checkpoints")
conf = SparkConf().setMaster('local').setAppName('connected component')

sc = SparkContext(conf=conf)
sqlcontext = SQLContext(sc)
sc.setCheckpointDir('~/Local Disk/checkpoints')

In [ ]:
## load dataset to storage function
def load_dataset(raw_data_path,categories,year,bucket):
    records=db.read_text(raw_data_path).map(lambda x:json.loads(x))
    for category in categories:
        docs = (records.filter(lambda x:category in x['categories']))
        extract_latest_version=lambda x:x['versions'][-1]["created"]
        if year!=None:
            docs=docs.filter(lambda x:int(extract_latest_version(x).split(" ")[3])>=year)

        get_metadata = lambda x: {'id': x['id'],
                        'title': x['title'],
                        'category':x['categories'],
                        'abstract':x['abstract'],
                        'version':x['versions'][-1]['created'],
                        'doi':x["doi"], 
                        'authors_parsed':x['authors_parsed']}

        data=docs.map(get_metadata).to_dataframe().compute()

        ## Creating authors fields by joining first and last nmes in authors_parsed columns.
        data['authors']=data['authors_parsed'].apply(lambda authors:[(" ".join(author)).strip() for author in authors])

        print("Number of Records Extracted for Category ",category, data.shape[0])

        data.to_json('gs://{}/arxiv_data_cs_field_category_{}.json'.format(bucket,category),orient="records")

In [40]:
## setting
year=2011
categories=['cs.AI','cs.CL','cs.CC','cs.CE','cs.CG','cs.GT','cs.CV','cs.CY','cs.CR','cs.DS'
           ,'cs.DB','cs.DL','cs.DM','cs.DC']
raw_data_path='gs://bigdata-homework2-question2-bucket/arxiv-metadata-oai-snapshot.json'
bucket = "bigdata-homework2-question2-bucket"

#load_dataset(raw_data_path=raw_data_path,categories=categories,year=year,bucket=bucket)

In [ ]:
## cs_field_total
categories=['cs.AI','cs.CL','cs.CC','cs.CE','cs.CG','cs.GT','cs.CV','cs.CY','cs.CR','cs.DS'
           ,'cs.DB','cs.DL','cs.DM','cs.DC','cs.DT','cs.FL','cs.GL','cs.GR','cs.AR','cs.HC'
           ,'cs.IR','cs.IT','cs.LG','cs.LO','cs.MS','cs,MA','cs.MM','cs.NI','cs.NE','cs.NA'
           ,'cs.OS','cs.OH','cs.PF','cs.PL','cs.RO','cs.SI','cs.SE','cs.SD','cs.SC','cs.SY']

records=db.read_text(raw_data_path).map(lambda x:json.loads(x))
docs = (records.filter(lambda x:any(ele in x['categories'] for ele in categories)==True))
extract_latest_version=lambda x:x['versions'][-1]["created"]
if year!=None:
    docs=docs.filter(lambda x:int(extract_latest_version(x).split(" ")[3])>=year)

get_metadata = lambda x: {'id': x['id'],
                        'title': x['title'],
                        'category':x['categories'],
                        'abstract':x['abstract'],
                        'version':x['versions'][-1]['created'],
                        'doi':x["doi"], 
                        'authors_parsed':x['authors_parsed']}

data=docs.map(get_metadata).to_dataframe().compute()

## Creating authors fields by joining first and last nmes in authors_parsed columns.
data['authors']=data['authors_parsed'].apply(lambda authors:[(" ".join(author)).strip() for author in authors])

print("Number of Records Extracted for cs field 2011 ",data.shape[0])

data.to_json('gs://{}/arxiv_data_cs_field_2011.json'.format(bucket),orient="records")

#### Creating a Co-Author Network

In [ ]:
for category in categories:
    rdd_test = sqlcontext.read.json('gs://bigdata-homework2-question2-bucket/arxiv_data_cs_field_category_{}.json'.format(category))
    authors = rdd_test.select('authors').rdd

    vertices = authors.flatMap(lambda x: [i for i in x[0]]).map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y)
    v=vertices.toDF(['id','article_num'])

    authors_pairs = authors.map(lambda x:list(permutations(x[0],2)))
    edges = authors_pairs.flatMap(lambda x:[i for i in x])
    e = edges.toDF(['src', 'dst'])


    g = GraphFrame(v, e)
    result = g.connectedComponents()
    
    cluster_amount = result.select('component').distinct().count()
    print("there are %d clusters in total" % cluster_amount)
    
    pg_result = g.pageRank(resetProbability=0.15,tol=0.01)
    userrank = pg_result.vertices.select('id','pagerank').orderBy('pagerank',ascending = False).head(10)

    ##component_id = result.where(result['id']==userrank[0].id).select('component').collect()[0].component
    local_result = g.pageRank(resetProbability=0.15, maxIter=20, sourceId=userrank[0].id)
    local_userrank = local_result.vertices.select('id','pagerank').orderBy('pagerank',ascending = False).head(10)
    local_userid = [row.id for row in local_userrank]

    #g2 = g.filterVertices("id='Levine Sergey'")
    g2 = g.filterVertices("id in {}".format(tuple(local_userid)))


    #(lambda x:any(ele in x['categories'] for ele in categories)==True))

    g2.vertices.show()
    
    nodes_list = [i for i in local_userid]
    with open('nodes_table_{}.csv'.format(category),'w',newline='') as f:
        write = csv.writer(f)
        write.writerow(['node'])
        write.writerow(nodes_list)

    edge_df = g2.edges.rdd.map(lambda x:(local_userid.index(x[0]),local_userid.index(x[1]))).toDF(['source','target'])
    edge_df.toPandas().to_csv('edges_table_{}.csv'.format(category),header=True,index=False)

there are 6265 clusters in total
+------------------+-----------+
|                id|article_num|
+------------------+-----------+
|     Abbeel Pieter|        124|
|     Levine Sergey|        146|
|      Finn Chelsea|         60|
|   Haarnoja Tuomas|          9|
|Rajeswaran Aravind|         14|
|         Yu Tianhe|         10|
|    Gupta Abhishek|         25|
|  Calandra Roberto|         15|
|    Ebert Frederik|          8|
|Eysenbach Benjamin|         14|
+------------------+-----------+

there are 3079 clusters in total
+--------------------+-----------+
|                  id|article_num|
+--------------------+-----------+
|         Hovy Eduard|         75|
|       Metze Florian|         42|
|       Neubig Graham|        163|
|    Nakamura Satoshi|         38|
|           Hu Junjie|         24|
|Anastasopoulos An...|         44|
|         Yang Yiming|         32|
|     Rijhwani Shruti|         13|
|          Wang Xinyi|         19|
|     Chaudhary Aditi|         13|
+---------------